In [28]:
import random
import math
import numpy as np
import matplotlib.pyplot as plt

In [29]:
class Neuron:
    def __init__(self, bias):
        self.bias = bias
        self.weights = []

    def calculate_output(self, inputs):
        self.inputs = inputs
        self.output = self.sigmoid(self.calculate_net_input())
        return self.output


    def calculate_net_input(self):
        total = 0
        for i in range(len(self.inputs)):
            total += self.inputs[i] * self.weights[i]
        return total + self.bias

    def sigmoid(self, net_input):
        return 1 / (1 + math.exp(-net_input))

    def relu(self, net_input):
        return max(0, net_input)

    def least_squares_error(self, target):
        return 0.5 * (target - self.output) ** 2

    # What's next?
    # Need to determine how much the neuron's total input has to change to move closer to the expected output
    # What is this value?
    # ∂E/∂zⱼ = ∂E/∂yⱼ * dyⱼ/dzⱼ

    def derivative_error_wrt_net_input(self, target):
        return self.derivative_error_wrt_output(target) * self.derivative_output_wrt_net_input()

    # Least Squares Error: 1/2 * (tⱼ - yⱼ)^2
    # The partial derivate of the error with respect to actual output then is calculated by:
    # ∂E/∂yⱼ = -(tⱼ - yⱼ)
    def derivative_error_wrt_output(self, target):
        return -1 * (target - self.output)

    # dyⱼ/dzⱼ = yⱼ * (1 - yⱼ)
    def derivative_output_wrt_net_input(self):
        return self.output * (1 - self.output)

    # The total net input is the weighted sum of all the inputs to the neuron and their respective weights:
    # zⱼ = netⱼ = x₁w₁ + x₂w₂ ...
    # The partial derivative of the total net input with respective to a given weight (with everything else held constant) then is:
    # = ∂zⱼ/∂wᵢ = some constant + 1 * xᵢw₁^(1-0) + some constant ... = xᵢ
    def derivative_net_input_wrt_weight(self, index):
        return self.inputs[index]

In [30]:
class Layer:
    """
    Creates a Layer in the network consisting of neurons/units.
    Each layer has one bias.


    Parameters:
    ------------
    N : Integer
        Number of neurons/units in the layer

    bias : Float
        bias of the layer

    neurons : List of objects of class Neuron
    """

    def __init__(self, N, bias):
        if bias:
            self.bias = bias
        else:
            self.bias = random.random()

        self.neurons = []

        for i in range(N):
            self.neurons.append(Neuron(self.bias))

    def feed_forward(self, inputs):
        outputs = []
        for neuron in self.neurons:
            outputs.append(neuron.calculate_output(inputs))
        return outputs

    def get_outputs(self):
        outputs = []
        for neuron in self.neurons:
            outputs.append(neuron.output)
        return outputs


In [31]:
class NeuralNetwork:
    learning_rate = 0.5

    def __init__(self, num_inputs, num_hidden, num_outputs, hidden_layer_weights = None, hidden_layer_bias = None, output_layer_bias = None, output_layer_weights = None):
        self.num_inputs = num_inputs
        self.hidden_layer = Layer(num_hidden, hidden_layer_bias)
        self.output_layer = Layer(num_outputs, output_layer_bias)
        # Let's initialize those weights we got there!
        self.initialize_weights_hidden_layer(hidden_layer_weights)
        self.initialize_weights_output_layer(output_layer_weights)


    def initialize_weights_hidden_layer(self, hidden_layer_weights):
        weight_num = 0
        for h in range(len(self.hidden_layer.neurons)):
            for i in range(self.num_inputs):
                if hidden_layer_weights:
                    self.hidden_layer.neurons[h].weights.append(hidden_layer_weights[weight_num])
                else:
                    self.hidden_layer.neurons[h].weights.append(random.random())
                weight_num += 1


    def initialize_weights_output_layer(self, output_layer_weights):
        weight_num = 0
        for h in range(len(self.output_layer.neurons)):
            for i in range(self.num_inputs):
                if output_layer_weights:
                    self.output_layer.neurons[h].weights.append(output_layer_weights[weight_num])
                else:
                    self.output_layer.neurons[h].weights.append(random.random())
                weight_num += 1

    def feed_forward(self, inputs):
        hidden_layer_outputs = self.hidden_layer.feed_forward(inputs)
        return self.output_layer.feed_forward(hidden_layer_outputs)

    # Using Stochastic Gradient Descent
    # Parameter "inputs" is the input to the network.
    # Parameter "target" is the groundtruth label for the given output.
    def train(self, inputs, target):
        self.feed_forward(inputs)

        # Step 1: Calculate output neurons derivative 
        # Calculate ∂E/∂z
        derivative_error_wrt_output_layer_net_input = [0] * len(self.output_layer.neurons)
        for i in range(len(self.output_layer.neurons)):
            #  Calculate ∂E/∂zⱼ
            derivative_error_wrt_output_layer_net_input[i] = self.output_layer.neurons[i].derivative_error_wrt_net_input(target[i])

        # Step 2: Calculate hidden neurons derivative
        # dE/dyⱼ = Σ ∂E/∂zⱼ * ∂z/∂yⱼ = Σ ∂E/∂zⱼ * wᵢⱼ
        derivative_error_wrt_hidden_layer_net_input = [0] * len(self.hidden_layer.neurons)
        for i in range(len(self.hidden_layer.neurons)):
            temp = 0
            for j in range(len(self.output_layer.neurons)):
                # what is temp?
                # 
                temp += derivative_error_wrt_output_layer_net_input[j] * self.output_layer.neurons[j].weights[i]

            derivative_error_wrt_hidden_layer_net_input[i] = temp * self.hidden_layer.neurons[i].derivative_output_wrt_net_input()

        # Step 3: Update output neurons weights
        for o in range(len(self.output_layer.neurons)):
            for weight_num in range(len(self.output_layer.neurons[o].weights)):
                # ∂Eⱼ/∂wᵢⱼ = ∂E/∂zⱼ * ∂zⱼ/∂wᵢⱼ
                derivative_error_wrt_weight = derivative_error_wrt_output_layer_net_input[o] * self.output_layer.neurons[o].derivative_net_input_wrt_weight(weight_num)
                # Δw = α * ∂Eⱼ/∂wᵢ
                self.output_layer.neurons[o].weights[weight_num] -= self.learning_rate * derivative_error_wrt_weight

        # Step 4: Update hidden neuron weights
        for h in range(len(self.hidden_layer.neurons)):
            for weight_num in range(len(self.hidden_layer.neurons[h].weights)):
                # ∂Eⱼ/∂wᵢ = ∂E/∂zⱼ * ∂zⱼ/∂wᵢ
                derivative_error_wrt_weight = derivative_error_wrt_hidden_layer_net_input[h] * self.hidden_layer.neurons[h].derivative_net_input_wrt_weight(weight_num)
                self.hidden_layer.neurons[h].weights[weight_num] -= self.learning_rate * derivative_error_wrt_weight

    def calculate_total_error(self, training_data):
        total_error = 0
        for t in range(len(training_data)):
            training_x, training_output = training_data[t]
            self.feed_forward(training_x)

            for o in range(len(training_output)):
                total_error += self.output_layer.neurons[o].least_squares_error(training_output[o])
        return total_error



In [42]:
nn = NeuralNetwork(2, 2, 2) # hidden_layer_weights=[0.15, 0.2, 0.25, 0.3], hidden_layer_bias=0.35, output_layer_weights=[0.4, 0.45, 0.5, 0.55], output_layer_bias=0.6)
for i in range(10000):
    nn.train([0, 1], [0, 1])
    nn.train([0, 0], [1, 0])
    nn.train([1, 0], [0, 1])
    nn.train([1, 1], [1, 0])
    
    print(i, round(nn.calculate_total_error([[[0, 1], [0, 1]]]),9))
    print(i, round(nn.calculate_total_error([[[0, 0], [1, 0]]]),9))
    print(i, round(nn.calculate_total_error([[[1, 0], [0, 1]]]),9))
    print(i, round(nn.calculate_total_error([[[1, 1], [1, 0]]]),9))

    #print(i, round(nn.calculate_total_error([[[0, 1], [0, 1]],[[0, 0], [1, 0], [[1, 0], [0, 1]],[[1, 1], [1, 0]]]), 9))
print(nn.feed_forward([0, 1]))
print(nn.feed_forward([0, 0]))
print(nn.feed_forward([1, 0]))
print(nn.feed_forward([1, 1]))

0 0.365816104
0 0.237013553
0 0.351072927
0 0.240574111
1 0.358202908
1 0.231937609
1 0.343827202
1 0.233215436
2 0.350434681
2 0.228008611
2 0.336534096
2 0.227368218
3 0.342649667
3 0.225153707
3 0.329315354
3 0.222954028
4 0.334993883
4 0.223262916
4 0.322293648
4 0.219841745
5 0.327604679
5 0.222202737
5 0.315579569
5 0.217865601
6 0.32059758
6 0.221829458
6 0.309262036
6 0.216842832
7 0.314058445
7 0.22200065
7 0.30340335
7 0.216589079
8 0.308041204
8 0.222583871
8 0.298038692
8 0.216930333
9 0.302570074
9 0.223462266
9 0.293178958
9 0.217710925
10 0.297644558
10 0.224537284
10 0.288815481
10 0.218797768
11 0.293245577
11 0.225729084
11 0.28492535
11 0.22008152
12 0.289341485
12 0.226975362
12 0.281476412
12 0.221475544
13 0.285893256
13 0.228229269
13 0.278431455
13 0.22291353
14 0.282858537
14 0.229456937
14 0.275751407
14 0.2243465
15 0.28019457
15 0.230635005
15 0.27339759
15 0.225739681
16 0.27786014
16 0.231748349
16 0.271333185
16 0.22706958
17 0.275816743
17 0.232788116
17

267 0.253341845
267 0.240243612
268 0.263996302
268 0.241993669
268 0.25332088
268 0.240244788
269 0.264009011
269 0.241998048
269 0.253299869
269 0.240245932
270 0.264021742
270 0.242002482
270 0.253278813
270 0.240247041
271 0.264034495
271 0.242006971
271 0.25325771
271 0.240248116
272 0.264047269
272 0.242011516
272 0.253236562
272 0.240249156
273 0.264060066
273 0.242016115
273 0.253215369
273 0.24025016
274 0.264072886
274 0.242020771
274 0.253194131
274 0.240251128
275 0.264085728
275 0.242025481
275 0.253172849
275 0.240252059
276 0.264098594
276 0.242030247
276 0.253151522
276 0.240252953
277 0.264111483
277 0.242035069
277 0.253130151
277 0.240253809
278 0.264124396
278 0.242039945
278 0.253108736
278 0.240254626
279 0.264137332
279 0.242044877
279 0.253087277
279 0.240255404
280 0.264150293
280 0.242049864
280 0.253065775
280 0.240256142
281 0.264163278
281 0.242054907
281 0.253044231
281 0.240256841
282 0.264176289
282 0.242060004
282 0.253022643
282 0.240257498
283 0.26418

517 0.247275392
517 0.23821006
518 0.268728703
518 0.244483095
518 0.247249607
518 0.238188326
519 0.268758148
519 0.244496079
519 0.247223814
519 0.238166455
520 0.268787717
520 0.244509061
520 0.247198012
520 0.238144449
521 0.268817409
521 0.244522039
521 0.247172202
521 0.238122306
522 0.268847225
522 0.244535014
522 0.247146383
522 0.238100026
523 0.268877166
523 0.244547985
523 0.247120556
523 0.238077609
524 0.268907232
524 0.244560952
524 0.247094719
524 0.238055055
525 0.268937424
525 0.244573914
525 0.247068873
525 0.238032362
526 0.268967742
526 0.244586872
526 0.247043018
526 0.238009532
527 0.268998188
527 0.244599824
527 0.247017153
527 0.237986563
528 0.26902876
528 0.244612771
528 0.246991278
528 0.237963455
529 0.269059462
529 0.244625712
529 0.246965393
529 0.237940208
530 0.269090292
530 0.244638647
530 0.246939498
530 0.237916821
531 0.269121251
531 0.244651576
531 0.246913593
531 0.237893294
532 0.26915234
532 0.244664499
532 0.246887677
532 0.237869626
533 0.26918

767 0.247143408
767 0.239160148
767 0.226697913
768 0.282302586
768 0.24715085
768 0.239109312
768 0.226613764
769 0.282397594
769 0.24715826
769 0.239058164
769 0.226529138
770 0.282493118
770 0.247165637
770 0.239006702
770 0.226444035
771 0.282589162
771 0.247172982
771 0.238954921
771 0.22635845
772 0.282685729
772 0.247180293
772 0.238902818
772 0.22627238
773 0.282782822
773 0.247187571
773 0.238850391
773 0.226185823
774 0.282880445
774 0.247194816
774 0.238797635
774 0.226098776
775 0.2829786
775 0.247202027
775 0.238744547
775 0.226011235
776 0.283077292
776 0.247209205
776 0.238691125
776 0.225923198
777 0.283176523
777 0.247216347
777 0.238637363
777 0.225834662
778 0.283276297
778 0.247223456
778 0.238583259
778 0.225745624
779 0.283376617
779 0.247230529
779 0.238528809
779 0.22565608
780 0.283477488
780 0.247237567
780 0.23847401
780 0.225566028
781 0.283578911
781 0.24724457
781 0.238418857
781 0.225475464
782 0.283680892
782 0.247251537
782 0.238363348
782 0.225384386
7

1017 0.245051447
1017 0.20174493
1017 0.183195125
1018 0.330673118
1018 0.245001836
1018 0.201414871
1018 0.182919569
1019 0.330974349
1019 0.244951602
1019 0.201082852
1019 0.182643564
1020 0.331275967
1020 0.244900743
1020 0.200748874
1020 0.182367124
1021 0.331577956
1021 0.244849254
1021 0.20041294
1021 0.182090263
1022 0.3318803
1022 0.244797132
1022 0.200075053
1022 0.181812995
1023 0.332182985
1023 0.244744374
1023 0.199735215
1023 0.181535334
1024 0.332485995
1024 0.244690976
1024 0.199393432
1024 0.181257294
1025 0.332789315
1025 0.244636935
1025 0.199049705
1025 0.180978889
1026 0.33309293
1026 0.244582248
1026 0.198704039
1026 0.180700134
1027 0.333396823
1027 0.244526911
1027 0.198356439
1027 0.180421044
1028 0.333700979
1028 0.244470921
1028 0.198006908
1028 0.180141633
1029 0.334005383
1029 0.244414275
1029 0.197655453
1029 0.179861916
1030 0.334310019
1030 0.24435697
1030 0.197302077
1030 0.179581908
1031 0.33461487
1031 0.244299003
1031 0.196946786
1031 0.179301625
1032

...

8979 0.435334912
8979 0.156529338
8979 0.000441362
8979 0.116269672
8980 0.435334897
8980 0.156529329
8980 0.000441302
8980 0.116269623
8981 0.435334882
8981 0.156529319
8981 0.000441241
8981 0.116269574
8982 0.435334867
8982 0.15652931
8982 0.00044118
8982 0.116269525
8983 0.435334852
8983 0.156529301
8983 0.00044112
8983 0.116269476
8984 0.435334836
8984 0.156529291
8984 0.000441059
8984 0.116269427
8985 0.435334821
8985 0.156529282
8985 0.000440999
8985 0.116269378
8986 0.435334806
8986 0.156529273
8986 0.000440938
8986 0.11626933
8987 0.435334791
8987 0.156529263
8987 0.000440877
8987 0.116269281
8988 0.435334776
8988 0.156529254
8988 0.000440817
8988 0.116269232
8989 0.435334761
8989 0.156529244
8989 0.000440756
8989 0.116269183
8990 0.435334746
8990 0.156529235
8990 0.000440696
8990 0.116269134
8991 0.435334731
8991 0.156529226
8991 0.000440635
8991 0.116269086
8992 0.435334716
8992 0.156529216
8992 0.000440575
8992 0.116269037
8993 0.4353347
8993 0.156529207
8993 0.000440514
899

9202 0.000428223
9202 0.116259077
9203 0.435331609
9203 0.156527299
9203 0.000428166
9203 0.116259031
9204 0.435331595
9204 0.15652729
9204 0.000428109
9204 0.116258985
9205 0.43533158
9205 0.156527282
9205 0.000428052
9205 0.116258939
9206 0.435331566
9206 0.156527273
9206 0.000427995
9206 0.116258893
9207 0.435331552
9207 0.156527264
9207 0.000427938
9207 0.116258846
9208 0.435331537
9208 0.156527255
9208 0.00042788
9208 0.1162588
9209 0.435331523
9209 0.156527246
9209 0.000427823
9209 0.116258754
9210 0.435331509
9210 0.156527238
9210 0.000427766
9210 0.116258708
9211 0.435331494
9211 0.156527229
9211 0.000427709
9211 0.116258662
9212 0.43533148
9212 0.15652722
9212 0.000427652
9212 0.116258616
9213 0.435331466
9213 0.156527211
9213 0.000427595
9213 0.11625857
9214 0.435331451
9214 0.156527202
9214 0.000427538
9214 0.116258524
9215 0.435331437
9215 0.156527194
9215 0.000427481
9215 0.116258478
9216 0.435331423
9216 0.156527185
9216 0.000427424
9216 0.116258432
9217 0.435331408
9217 

9432 0.435328404
9432 0.156525344
9432 0.000415449
9432 0.116248761
9433 0.43532839
9433 0.156525336
9433 0.000415395
9433 0.116248718
9434 0.435328376
9434 0.156525328
9434 0.000415341
9434 0.116248674
9435 0.435328363
9435 0.156525319
9435 0.000415287
9435 0.116248631
9436 0.435328349
9436 0.156525311
9436 0.000415233
9436 0.116248587
9437 0.435328335
9437 0.156525303
9437 0.000415179
9437 0.116248544
9438 0.435328322
9438 0.156525295
9438 0.000415126
9438 0.1162485
9439 0.435328308
9439 0.156525286
9439 0.000415072
9439 0.116248457
9440 0.435328294
9440 0.156525278
9440 0.000415018
9440 0.116248413
9441 0.435328281
9441 0.15652527
9441 0.000414964
9441 0.11624837
9442 0.435328267
9442 0.156525262
9442 0.00041491
9442 0.116248326
9443 0.435328254
9443 0.156525253
9443 0.000414857
9443 0.116248283
9444 0.43532824
9444 0.156525245
9444 0.000414803
9444 0.116248239
9445 0.435328226
9445 0.156525237
9445 0.000414749
9445 0.116248196
9446 0.435328213
9446 0.156525229
9446 0.000414695
9446

9595 0.43532622
9595 0.156524026
9595 0.000406837
9595 0.116241799
9596 0.435326207
9596 0.156524018
9596 0.000406785
9596 0.116241757
9597 0.435326193
9597 0.15652401
9597 0.000406733
9597 0.116241715
9598 0.43532618
9598 0.156524002
9598 0.000406682
9598 0.116241673
9599 0.435326167
9599 0.156523994
9599 0.00040663
9599 0.116241631
9600 0.435326154
9600 0.156523986
9600 0.000406578
9600 0.11624159
9601 0.435326141
9601 0.156523979
9601 0.000406527
9601 0.116241548
9602 0.435326128
9602 0.156523971
9602 0.000406475
9602 0.116241506
9603 0.435326115
9603 0.156523963
9603 0.000406423
9603 0.116241464
9604 0.435326101
9604 0.156523955
9604 0.000406372
9604 0.116241422
9605 0.435326088
9605 0.156523947
9605 0.00040632
9605 0.116241381
9606 0.435326075
9606 0.156523939
9606 0.000406268
9606 0.116241339
9607 0.435326062
9607 0.156523931
9607 0.000406217
9607 0.116241297
9608 0.435326049
9608 0.156523923
9608 0.000406165
9608 0.116241255
9609 0.435326036
9609 0.156523915
9609 0.000406113
960

9827 0.156522247
9827 0.000395164
9827 0.11623235
9828 0.43532323
9828 0.156522239
9828 0.000395115
9828 0.116232311
9829 0.435323217
9829 0.156522232
9829 0.000395066
9829 0.116232271
9830 0.435323205
9830 0.156522224
9830 0.000395017
9830 0.116232231
9831 0.435323192
9831 0.156522217
9831 0.000394968
9831 0.116232192
9832 0.43532318
9832 0.15652221
9832 0.000394919
9832 0.116232152
9833 0.435323167
9833 0.156522202
9833 0.000394871
9833 0.116232113
9834 0.435323155
9834 0.156522195
9834 0.000394822
9834 0.116232073
9835 0.435323142
9835 0.156522187
9835 0.000394773
9835 0.116232034
9836 0.43532313
9836 0.15652218
9836 0.000394724
9836 0.116231994
9837 0.435323117
9837 0.156522172
9837 0.000394675
9837 0.116231955
9838 0.435323105
9838 0.156522165
9838 0.000394627
9838 0.116231915
9839 0.435323092
9839 0.156522158
9839 0.000394578
9839 0.116231875
9840 0.43532308
9840 0.15652215
9840 0.000394529
9840 0.116231836
9841 0.435323068
9841 0.156522143
9841 0.00039448
9841 0.116231796
9842 0

In [43]:
nn = NeuralNetwork(2, 2, 1) # hidden_layer_weights=[0.15, 0.2, 0.25, 0.3], hidden_layer_bias=0.35, output_layer_weights=[0.4, 0.45, 0.5, 0.55], output_layer_bias=0.6)
for i in range(10000):
    nn.train([0, 1], [1])
    nn.train([0, 0], [0])
    nn.train([1, 0], [1])
    nn.train([1, 1], [0])
    
    print(i, round(nn.calculate_total_error([[[0, 1], [1]]]),9))
    print(i, round(nn.calculate_total_error([[[0, 0], [0]]]),9))
    print(i, round(nn.calculate_total_error([[[1, 0], [1]]]),9))
    print(i, round(nn.calculate_total_error([[[1, 1], [0]]]),9))

    #print(i, round(nn.calculate_total_error([[[0, 1], [0, 1]],[[0, 0], [1, 0], [[1, 0], [0, 1]],[[1, 1], [1, 0]]]), 9))
print(nn.feed_forward([0, 1]))
print(nn.feed_forward([0, 0]))
print(nn.feed_forward([1, 0]))
print(nn.feed_forward([1, 1]))

0 0.009436502
0 0.361838104
0 0.007554489
0 0.392688802
1 0.010491984
1 0.355959247
1 0.008524195
1 0.385709122
2 0.0116771
2 0.349755006
2 0.009630444
2 0.378264194
3 0.013005384
3 0.343224116
3 0.010890363
3 0.370349369
4 0.014490526
4 0.336370709
4 0.012321895
4 0.361967709
5 0.016145903
5 0.329205349
5 0.013943307
5 0.353131445
6 0.01798398
6 0.321745973
6 0.015772504
6 0.343863271
7 0.020015591
7 0.314018627
7 0.017826159
7 0.334197331
8 0.02224913
8 0.306057869
8 0.020118654
8 0.324179738
9 0.02468969
9 0.297906722
9 0.022660904
9 0.313868469
10 0.027338225
10 0.289616081
10 0.025459136
10 0.30333252
11 0.030190825
11 0.281243521
11 0.028513739
11 0.292650274
12 0.033238192
12 0.272851526
12 0.031818338
12 0.281907109
13 0.036465405
13 0.264505241
13 0.035359212
13 0.271192371
14 0.039852049
14 0.256269918
14 0.039115191
14 0.260595936
15 0.043372722
15 0.248208281
15 0.043058098
15 0.250204649
16 0.046997915
16 0.240378068
16 0.047153722
16 0.240098954
17 0.050695179
17 0.232829

277 0.13495864
277 0.117381063
278 0.118478119
278 0.128465813
278 0.134963101
278 0.117396653
279 0.118477881
279 0.128443108
279 0.134967528
279 0.117412154
280 0.118477656
280 0.128420523
280 0.134971921
280 0.117427566
281 0.118477446
281 0.128398056
281 0.13497628
281 0.11744289
282 0.118477249
282 0.128375708
282 0.134980607
282 0.117458126
283 0.118477065
283 0.128353478
283 0.134984902
283 0.117473273
284 0.118476894
284 0.128331365
284 0.134989165
284 0.117488332
285 0.118476736
285 0.128309369
285 0.134993397
285 0.117503302
286 0.118476591
286 0.128287489
286 0.134997599
286 0.117518185
287 0.118476459
287 0.128265725
287 0.13500177
287 0.11753298
288 0.118476338
288 0.128244076
288 0.135005911
288 0.117547686
289 0.118476229
289 0.128222541
289 0.135010023
289 0.117562305
290 0.118476132
290 0.12820112
290 0.135014106
290 0.117576837
291 0.118476047
291 0.128179813
291 0.135018161
291 0.11759128
292 0.118475973
292 0.128158619
292 0.135022188
292 0.117605636
293 0.118475909

513 0.135944693
513 0.118850122
514 0.11830966
514 0.125488342
514 0.135951615
514 0.118847955
515 0.118306494
515 0.125482899
515 0.13595858
515 0.118845724
516 0.118303292
516 0.125477498
516 0.135965589
516 0.11884343
517 0.118300054
517 0.125472138
517 0.135972641
517 0.118841072
518 0.118296782
518 0.125466819
518 0.135979738
518 0.11883865
519 0.118293474
519 0.125461541
519 0.135986879
519 0.118836165
520 0.11829013
520 0.125456305
520 0.135994064
520 0.118833616
521 0.11828675
521 0.125451109
521 0.136001294
521 0.118831003
522 0.118283335
522 0.125445953
522 0.136008569
522 0.118828327
523 0.118279883
523 0.125440838
523 0.136015889
523 0.118825588
524 0.118276395
524 0.125435763
524 0.136023254
524 0.118822785
525 0.118272871
525 0.125430728
525 0.136030665
525 0.118819919
526 0.11826931
526 0.125425733
526 0.136038121
526 0.118816989
527 0.118265712
527 0.125420778
527 0.136045623
527 0.118813997
528 0.118262078
528 0.125415862
528 0.136053171
528 0.118810941
529 0.118258406

763 0.12503361
763 0.139510933
763 0.116340924
764 0.116077716
764 0.125034038
764 0.139534559
764 0.116322577
765 0.11606161
765 0.125034474
765 0.139558276
765 0.116304157
766 0.116045434
766 0.125034916
766 0.139582085
766 0.116285664
767 0.11602919
767 0.125035366
767 0.139605986
767 0.116267099
768 0.116012877
768 0.125035823
768 0.139629979
768 0.11624846
769 0.115996495
769 0.125036287
769 0.139654063
769 0.116229748
770 0.115980044
770 0.125036757
770 0.13967824
770 0.116210962
771 0.115963524
771 0.125037235
771 0.13970251
771 0.116192104
772 0.115946934
772 0.125037719
772 0.139726872
772 0.116173171
773 0.115930274
773 0.125038209
773 0.139751327
773 0.116154165
774 0.115913544
774 0.125038705
774 0.139775875
774 0.116135085
775 0.115896745
775 0.125039208
775 0.139800517
775 0.116115931
776 0.115879876
776 0.125039717
776 0.139825252
776 0.116096703
777 0.115862936
777 0.125040232
777 0.139850082
777 0.116077401
778 0.115845926
778 0.125040752
778 0.139875005
778 0.11605802

1006 0.109447688
1007 0.109804519
1007 0.125038553
1007 0.148482779
1007 0.109408177
1008 0.109767735
1008 0.125036773
1008 0.148534495
1008 0.109368571
1009 0.109730857
1009 0.125034967
1009 0.14858634
1009 0.109328869
1010 0.109693883
1010 0.125033134
1010 0.148638314
1010 0.109289072
1011 0.109656816
1011 0.125031275
1011 0.148690415
1011 0.109249181
1012 0.109619654
1012 0.12502939
1012 0.148742646
1012 0.109209194
1013 0.109582397
1013 0.125027477
1013 0.148795004
1013 0.109169113
1014 0.109545046
1014 0.125025538
1014 0.148847491
1014 0.109128937
1015 0.109507601
1015 0.125023572
1015 0.148900106
1015 0.109088667
1016 0.109470062
1016 0.125021579
1016 0.148952849
1016 0.109048302
1017 0.109432428
1017 0.125019559
1017 0.149005719
1017 0.109007842
1018 0.109394701
1018 0.125017511
1018 0.149058718
1018 0.108967288
1019 0.109356879
1019 0.125015436
1019 0.149111844
1019 0.10892664
1020 0.109318964
1020 0.125013333
1020 0.149165098
1020 0.108885897
1021 0.109280955
1021 0.125011203


...

8933 0.048002009
8933 0.000387234
8933 0.128187385
8933 0.122719584
8934 0.048001863
8934 0.000387173
8934 0.128187474
8934 0.122719381
8935 0.048001718
8935 0.000387112
8935 0.128187563
8935 0.122719178
8936 0.048001573
8936 0.000387051
8936 0.128187652
8936 0.122718975
8937 0.048001428
8937 0.00038699
8937 0.128187741
8937 0.122718773
8938 0.048001283
8938 0.000386929
8938 0.12818783
8938 0.12271857
8939 0.048001138
8939 0.000386868
8939 0.128187919
8939 0.122718367
8940 0.048000992
8940 0.000386807
8940 0.128188008
8940 0.122718164
8941 0.048000847
8941 0.000386746
8941 0.128188097
8941 0.122717962
8942 0.048000702
8942 0.000386685
8942 0.128188186
8942 0.122717759
8943 0.048000558
8943 0.000386624
8943 0.128188275
8943 0.122717557
8944 0.048000413
8944 0.000386564
8944 0.128188364
8944 0.122717354
8945 0.048000268
8945 0.000386503
8945 0.128188453
8945 0.122717152
8946 0.048000123
8946 0.000386442
8946 0.128188542
8946 0.12271695
8947 0.047999978
8947 0.000386381
8947 0.128188631
8

9136 0.047973276
9136 0.000375211
9136 0.128205112
9136 0.122679363
9137 0.047973138
9137 0.000375154
9137 0.128205197
9137 0.122679169
9138 0.047973
9138 0.000375097
9138 0.128205283
9138 0.122678976
9139 0.047972863
9139 0.000375039
9139 0.128205368
9139 0.122678783
9140 0.047972725
9140 0.000374982
9140 0.128205454
9140 0.12267859
9141 0.047972587
9141 0.000374925
9141 0.128205539
9141 0.122678396
9142 0.04797245
9142 0.000374867
9142 0.128205625
9142 0.122678203
9143 0.047972312
9143 0.00037481
9143 0.12820571
9143 0.12267801
9144 0.047972174
9144 0.000374753
9144 0.128205796
9144 0.122677817
9145 0.047972037
9145 0.000374695
9145 0.128205881
9145 0.122677624
9146 0.047971899
9146 0.000374638
9146 0.128205966
9146 0.122677431
9147 0.047971762
9147 0.000374581
9147 0.128206052
9147 0.122677238
9148 0.047971624
9148 0.000374524
9148 0.128206137
9148 0.122677045
9149 0.047971487
9149 0.000374466
9149 0.128206223
9149 0.122676852
9150 0.047971349
9150 0.000374409
9150 0.128206308
9150 

9386 0.047939872
9386 0.000361371
9386 0.128225973
9386 0.122632386
9387 0.047939743
9387 0.000361317
9387 0.128226054
9387 0.122632203
9388 0.047939613
9388 0.000361264
9388 0.128226136
9388 0.122632021
9389 0.047939484
9389 0.000361211
9389 0.128226217
9389 0.122631838
9390 0.047939355
9390 0.000361157
9390 0.128226299
9390 0.122631656
9391 0.047939225
9391 0.000361104
9391 0.12822638
9391 0.122631473
9392 0.047939096
9392 0.000361051
9392 0.128226461
9392 0.122631291
9393 0.047938967
9393 0.000360998
9393 0.128226542
9393 0.122631109
9394 0.047938837
9394 0.000360944
9394 0.128226624
9394 0.122630926
9395 0.047938708
9395 0.000360891
9395 0.128226705
9395 0.122630744
9396 0.047938579
9396 0.000360838
9396 0.128226786
9396 0.122630562
9397 0.04793845
9397 0.000360785
9397 0.128226867
9397 0.12263038
9398 0.047938321
9398 0.000360731
9398 0.128226949
9398 0.122630198
9399 0.047938192
9399 0.000360678
9399 0.12822703
9399 0.122630016
9400 0.047938063
9400 0.000360625
9400 0.128227111
9

9629 0.122589223
9630 0.047909203
9630 0.000348791
9630 0.128245363
9630 0.12258905
9631 0.047909081
9631 0.000348741
9631 0.128245441
9631 0.122588877
9632 0.047908959
9632 0.000348692
9632 0.128245518
9632 0.122588704
9633 0.047908837
9633 0.000348642
9633 0.128245596
9633 0.122588532
9634 0.047908715
9634 0.000348592
9634 0.128245673
9634 0.122588359
9635 0.047908593
9635 0.000348542
9635 0.128245751
9635 0.122588186
9636 0.047908471
9636 0.000348493
9636 0.128245828
9636 0.122588014
9637 0.047908349
9637 0.000348443
9637 0.128245906
9637 0.122587841
9638 0.047908227
9638 0.000348393
9638 0.128245983
9638 0.122587669
9639 0.047908106
9639 0.000348344
9639 0.128246061
9639 0.122587496
9640 0.047907984
9640 0.000348294
9640 0.128246138
9640 0.122587324
9641 0.047907862
9641 0.000348244
9641 0.128246216
9641 0.122587152
9642 0.04790774
9642 0.000348195
9642 0.128246293
9642 0.122586979
9643 0.047907619
9643 0.000348145
9643 0.128246371
9643 0.122586807
9644 0.047907497
9644 0.000348095

9823 0.000339431
9823 0.128260062
9823 0.122556402
9824 0.04788607
9824 0.000339384
9824 0.128260136
9824 0.122556237
9825 0.047885953
9825 0.000339337
9825 0.128260211
9825 0.122556071
9826 0.047885837
9826 0.00033929
9826 0.128260286
9826 0.122555906
9827 0.04788572
9827 0.000339242
9827 0.128260361
9827 0.12255574
9828 0.047885604
9828 0.000339195
9828 0.128260435
9828 0.122555575
9829 0.047885488
9829 0.000339148
9829 0.12826051
9829 0.122555409
9830 0.047885371
9830 0.000339101
9830 0.128260585
9830 0.122555244
9831 0.047885255
9831 0.000339054
9831 0.128260659
9831 0.122555079
9832 0.047885138
9832 0.000339007
9832 0.128260734
9832 0.122554913
9833 0.047885022
9833 0.00033896
9833 0.128260809
9833 0.122554748
9834 0.047884906
9834 0.000338912
9834 0.128260883
9834 0.122554583
9835 0.04788479
9835 0.000338865
9835 0.128260958
9835 0.122554418
9836 0.047884673
9836 0.000338818
9836 0.128261032
9836 0.122554252
9837 0.047884557
9837 0.000338771
9837 0.128261107
9837 0.122554087
9838